## Segment Anything Model (SAM) - EDX: live workflow
- Acquire an image using HAADF detector
- Choose a point 
- Acquire EDX detector signal
#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 2nd May 2025
#### edited - 
   

In [ ]:
from stemOrchestrator.logging_config   import setup_logging
data_folder  = "."
out_path = data_folder
setup_logging(out_path=out_path) 


In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from autoscript_tem_microscope_client.enumerations import EdsDetectorType
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging
plot = plt
from typing import Dict
import os

import numpy as np
import json
from pathlib import Path

In [ ]:
def acquire_and_plot_edx(image_data: np.ndarray, particle_idx: int, x: float, y: float,
                         eds_detector_name: EdsDetectorType, edx_exposure: float, out_path: str):
    """Acquire EDS spectrum at (x, y), overlay on HAADF, and save everything."""
    print(f"Acquiring EDX at normalized position: ({x}, {y}) for particle {particle_idx}")

    directory = os.path.join(out_path, f"manual_particle_{particle_idx}")
    os.makedirs(directory, exist_ok=True)

    # Configure and acquire EDX
    settings = tf_acquisition.configure_eds_settings(
        eds_detector_name,
        dispersion=5,
        shaping_time=3e-6,
        exposure_time=edx_exposure,
    )
    tf_acquisition.move_paused_beam(x, y)
    tf_acquisition.unblank_beam()
    spec = tf_acquisition.acquire_eds(settings)
    tf_acquisition.blank_beam()

    # Plot image with red cross at location
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    axs[0].imshow(image_data, cmap='gray')
    axs[0].scatter(x * image_data.shape[1], y * image_data.shape[0], c='r', s=100, marker='x')
    axs[0].set_title(f'HAADF with EDX Position ({x:.2f}, {y:.2f})')
    axs[0].axis("off")

    axs[1].plot(np.arange(len(spec)) * 5 / 1000, spec, label="EDS Spectrum")
    axs[1].set_xlabel("Energy (keV)")
    axs[1].set_ylabel("Counts")
    axs[1].set_title("EDS Spectrum")
    axs[1].legend()

    plt.tight_layout()
    fig.savefig(f"{directory}/haadf_edx_combined.png", dpi=300)
    plt.close()

    np.save(f"{directory}/spectrum.npy", spec)

    metadata = {
        "position_normalized": (x, y),
        "exposure_time": edx_exposure,
    }
    with open(f"{directory}/metadata.json", "w") as f:
        json.dump(metadata, f, indent=2)

    print(f"Saved EDX spectrum and plot for particle {particle_idx}.")


def main(config: Dict) -> None:
    ip = config["ip"]
    port = config["port"]
    haadf_exposure = config["haadf_exposure"]
    haadf_resolution = config["haadf_resolution"]
    out_path = config["out_path"]
    edx_exposure = config["edx_exposure"]
    eds_detector_name = config["eds_detector_name"]

    setup_logging(out_path)

    microscope = TemMicroscopeClient()
    global tf_acquisition
    tf_acquisition = TFacquisition(microscope=microscope, offline=True)

    # Step 1: Acquire HAADF
    haadf_np_array, haadf_tiff_name = tf_acquisition.acquire_haadf(
        exposure=haadf_exposure,
        resolution=haadf_resolution
    )
    HAADF_tiff_to_png(haadf_tiff_name)

    # Step 2: Ask user for positions
    print("\n--- Enter normalized (x, y) positions for EDX acquisition ---")
    print("Format: 0.5,0.6 or press Enter to stop")

    particle_idx = 0
    while True:
        inp = input(f"Enter position {particle_idx + 1} (or press Enter to finish): ").strip()
        if inp == "":
            break
        try:
            x, y = map(float, inp.split(","))
            if not (0 <= x <= 1 and 0 <= y <= 1):
                print("Coordinates must be normalized between 0 and 1. Try again.")
                continue
        except ValueError:
            print("Invalid format. Use: 0.5,0.6")
            continue

        acquire_and_plot_edx(haadf_np_array, particle_idx, x, y, eds_detector_name, edx_exposure, out_path)
        particle_idx += 1

    print("All EDX acquisitions completed.")



In [ ]:

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF_sim")
            port = port or secret.get("port_TF_sim")

if not ip:
    ip = input("Enter microscope IP: ")
if not port:
    port = input("Enter microscope Port: ")
port = int(port)

config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,
    "haadf_resolution": 512,
    "edx_exposure": 0.05,
    "eds_detector_name": EdsDetectorType.SUPER_X,
    "out_path": "."
}

main(config=config)
